##Yolov4 Data Preprocessing
Dataset Link : https://www.kaggle.com/datasets/andrewmvd/car-plate-detection?resource=download

This dataset contains 433 images with bounding box annotations of the car license plates within the image.
Annotations are provided in the PASCAL VOC format.

First the dataset has to annotated in YOLO supported format as the data is in PASCAL VOC format.

In Pascal VOC format the annotation will be labeld as Xmin, Xmax, Ymin, Ymax for the bounding box but YOLO needs the centre co-ordinate of the bounding box. So the annotaions needs to formatted as Xcentre, Ycentre, Width and Height co-ordinates. This is the co-ordinates for the centre point of the bounding box.
For annotation and Data Pre-processing referneced source is : https://campusx1040.graphy.com/s/courses/663d30056041c83f3f79b03f/take

####Formula for YOLO Annotations

* **Xcentre = (Xmin + Xmax) / 2**
* **Ycentre = (Ymin + Ymax) / 2**
* **Height =  (Ymax - Ymin)**
* **Width =   (Xmax - Xmin)**

But Yolo expects normalised data as its input. Hence the input co-ordinates needs to be normalised before feeding it to YOLO.

In [1]:
## Loading and installing all necessary libraries for data pre-processing and data annotations in YOLO format structure for model training
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from glob import glob
import xml.etree.ElementTree as xet
import cv2







In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import os

# Define folder path
folder_path = '/content/drive/MyDrive/Number_Plate'

# List files in the folder
files = os.listdir(folder_path)
print("Files in the folder:")
print(files)


Files in the folder:
['.DS_Store', 'images', 'annotations', 'labels']


In [16]:
## Regex Function to sort and filter the annotated data and images in the datasets folder
## The data inside the dataset folders are not alligned to the original structure

import re

def string_formating(filename):

  """
  The function helps to extract the digit integer from the filename
  And if there is no digits exist it returns a value '0'

  """

  match = re.search(r'(\d+)', filename)

  if match:
        return int(match.group(0))

  else:
        return 0


In [19]:
## Testing the RegEx function for its application on various examples
print(string_formating('Abc.123.txt'))
print(string_formating('abc.txt'))


123
0


In [47]:
##Creating a dictionary to store the values of Necessary Co-ordinates and information about the image

labels_dict = dict(

            img_path  =    [],
            xmax      =    [],
            xmin      =    [],
            ymin      =    [],
            ymax      =    [],
            width     =    [],
            height    =    []
)




##Creating .xml file and store all the annoted information using glob library

xml_files = glob(f'{folder_path}/annotations/*.xml')


## Sort all the shufle file path for preprcessing

for files_xml in sorted(xml_files, key = string_formating):

  ## This will parse the xml file
  info = xet.parse(files_xml)
  root = info.getroot()
  print(info)
  print(root)

  ## From object element of the annotated folder
  ## Extract the co-ordinates of the Bouniding Box

  each_object = root.find('object')
  labels_info = each_object.find('bndbox')

  xmin = int(labels_info.find('xmin').text)   ## Co-ordinate for Xmin for first BBox for first image
  xmax = int(labels_info.find('xmax').text)   ## Co-ordinate for Xmax for first BBox for first image
  ymin = int(labels_info.find('ymin').text)   ## Co-ordinate for Ymin for first BBox for first image
  ymax = int(labels_info.find('ymax').text)   ## Co-ordinate for Ymin for first BBox for first image


  img_name = root.find('filename').text
  img_path = os.path.join(folder_path, 'images', img_name)

  height , width, _  = cv2.imread(img_path).shape


  ## Append all the information retrieved from the folder to the dictionary lables_dict

  labels_dict['img_path'].append(img_path)
  labels_dict['xmin'].append(xmin)
  labels_dict['xmax'].append(xmax)
  labels_dict['ymin'].append(ymin)
  labels_dict['ymax'].append(ymax)
  labels_dict['width'].append(width)
  labels_dict['height'].append(height)






## Creating a DataFrame and storing all the values

all_data = pd.DataFrame(labels_dict)

<Element 'annotation' at 0x78f1e3356c00>
<Element 'annotation' at 0x78f1e3356fc0>
<Element 'annotation' at 0x78f1ee8acf40>
<Element 'annotation' at 0x78f1e3354db0>
<Element 'annotation' at 0x78f1e33546d0>
<Element 'annotation' at 0x78f1e3356200>
<Element 'annotation' at 0x78f1e33f6840>
<Element 'annotation' at 0x78f1e3354860>
<Element 'annotation' at 0x78f1eea096c0>
<Element 'annotation' at 0x78f1ee8acf40>
<Element 'annotation' at 0x78f1ee89a6b0>
<Element 'annotation' at 0x78f1ee975c60>
<Element 'annotation' at 0x78f1eea096c0>
<Element 'annotation' at 0x78f1eea711c0>
<Element 'annotation' at 0x78f1ef437f10>
<Element 'annotation' at 0x78f1ee9761b0>
<Element 'annotation' at 0x78f1ee8aed40>
<Element 'annotation' at 0x78f1eea09940>
<Element 'annotation' at 0x78f1ee89a6b0>
<Element 'annotation' at 0x78f1ee899120>
<Element 'annotation' at 0x78f1e299f3d0>
<Element 'annotation' at 0x78f1ee975c60>
<Element 'annotation' at 0x78f1ee8aed40>
<Element 'annotation' at 0x78f1ee899120>
<Element 'annota

In [48]:
all_data

,img_path,xmax,xmin,ymin,ymax,width,height
0,/content/drive/MyDrive/Number_Plate/images/Car...,419,226,125,173,500,268
1,/content/drive/MyDrive/Number_Plate/images/Car...,262,134,128,160,400,248
2,/content/drive/MyDrive/Number_Plate/images/Car...,270,229,176,193,400,400
3,/content/drive/MyDrive/Number_Plate/images/Car...,261,142,128,157,400,225
4,/content/drive/MyDrive/Number_Plate/images/Car...,503,156,82,253,590,350
...,...,...,...,...,...,...,...
428,/content/drive/MyDrive/Number_Plate/images/Car...,258,142,128,157,400,225
429,/content/drive/MyDrive/Number_Plate/images/Car...,208,86,166,195,301,400
430,/content/drive/MyDrive/Number_Plate/images/Car...,116,38,159,197,400,225
431,/content/drive/MyDrive/Number_Plate/images/Car...,343,55,82,147,400,192


### Saving the file in the Number Plate folder
The folder contains YOLO supported file format for co-ordinate and class for each images. All files are stored in .txt file inside Lables folder.

In [49]:
## Folder to store YOLO-style labels
yolo_label_dir = "/content/drive/MyDrive/Number_Plate/labels"
os.makedirs(yolo_label_dir, exist_ok=True)

## Class ID for license plates
class_id = 0

### This will store the labels of each co-ordinate for bounding box in text file
### The data will be stores at a different folder named labels
### P.S. The folder is saved in Google drive location path

## Group by image path in case multiple boxes are present
grouped = all_data.groupby('img_path')

for img_path, group in grouped:
    base_name = os.path.splitext(os.path.basename(img_path))[0]
    label_path = os.path.join(yolo_label_dir, base_name + ".txt")

    lines = []

    for _, row in group.iterrows():

        ## Yolo Expects a Normalised co-ordinates
        ## Devided with its appropriate values to make the Labels data Normalised

        x_center = ((row['xmin'] + row['xmax']) / 2) / row['width']  ## Dividing by ['Width'] to Normalise the data
        y_center = ((row['ymin'] + row['ymax']) / 2) / row['height'] ## Dividing by ['height'] to Normalise the data
        box_width = (row['xmax'] - row['xmin']) / row['width']       ## Dividing by ['Width'] to Normalise the data
        box_height = (row['ymax'] - row['ymin']) / row['height']     ## Dividing by ['height'] to Normalise the data

        yolo_line = f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}"
        lines.append(yolo_line)

    with open(label_path, "w") as f:
        f.write("\n".join(lines))


### Displaying first few lines of labels folder
It should return the data point in the format of [0 0.645000 0.555970 0.386000 0.179104] where 0 denotes the class here. For this problem statement the number of class is 1 as we are finding the Bounding Box for License Plate class.

In [6]:
## Displaying the first few files of labels folder
## Cross check if all the data stored in correct format or not
## This code has been refernced from ChatGPT for displaying the first 5 rows in lables folder
## It should display the co-ordinate in [0 0.645000 0.555970 0.386000 0.179104]
## Here the first digit will be always 0 as that denotes the numebr of class
## For this problem statement the number of class is 1 which denotes the bbox for Number Plate


labels_dir = "/content/drive/MyDrive/Number_Plate/labels"

# Check if path exists
if not os.path.exists(labels_dir):
    print(f"❌ Path does not exist: {labels_dir}")
else:
    # List all .txt files in labels directory
    label_files = [f for f in os.listdir(labels_dir) if f.endswith(".txt")]

    print(f"📂 Found {len(label_files)} label files.")

    # Show first 5 files
    for file_name in label_files[:5]:
        print(f"\n📄 File: {file_name}")
        with open(os.path.join(labels_dir, file_name), "r") as f:
            lines = f.readlines()
            for line in lines:
                print("  →", line.strip())


📂 Found 433 label files.

📄 File: Cars0.txt
  → 0 0.645000 0.555970 0.386000 0.179104

📄 File: Cars1.txt
  → 0 0.495000 0.580645 0.320000 0.129032

📄 File: Cars10.txt
  → 0 0.553750 0.340000 0.407500 0.635556

📄 File: Cars100.txt
  → 0 0.486250 0.458801 0.097500 0.063670

📄 File: Cars101.txt
  → 0 0.508750 0.703333 0.182500 0.060000


## Next Step
###### 1. YOLOv4 Architecture
###### 2. Modified Architecture with vision Transformers and various Attention Mechanism
###### 3. Train the model in Hyperion
###### 4. Log the metrics in wandb
###### 5. Save the model
###### 6. Inference
###### 7. Deployement(Testing)